# Função Gama


A Função Gama utilizada nesse programa tem como intuito receber os dados de posições e de doses de arquivos .csv, para tratá-los corretamente e aplicá-los em uma função matemática. 

Assim, seu retorno nos devolve os pontos como "aprovados" ou "reprovados", no sentido de verificar em quais pontos as doses são qualificadas, mediante comparação entre as calculadas e medidas.

In [ ]:
# Valores de input da diferença percentual de doses e DTA
delta_Dm = float(input("Insira aqui a diferença percentual de dose: "))
delta_dM = float(input("Insira agora o valor de DTA: "))

In [ ]:
import pandas as pd
import numpy as np

def gamma_index_function(rc, rm, Dc, Dm, max_Dm):
    '''
    Função Gama que será utilizada ao longo do código,
    recebendo como parâmetros as medidas de posições (cm) 
    e doses (cGy) calculadas e medidas. 
    '''
    r_diff_x = np.sqrt(rc[0]**2 + rc[1]**2)
    r_diff_y = np.sqrt(rm[0]**2 + rm[1]**2)
    r_diff = abs(r_diff_x - r_diff_y) # cálculo do módulo das posições
    D_diff = abs(Dc - Dm)/(max_Dm) # diferença entre doses
    
    F_g = np.sqrt((r_diff/delta_dM)**2 + (D_diff/delta_Dm)**2) # cálculo da função gamma

    return F_g

def position_module(row, max_Dm):
    '''
    Função para calcular o módulo de posição, que chama a
    função Gamma já passando os parâmetros necessários.
    '''
    rm_x = row['Ponto Medido X (rm_x)']
    rm_y = row['Ponto Medido Y (rm_y)']
    rc_x = row['Ponto Calculado X (rc_x)']
    rc_y = row['Ponto Calculado Y (rc_y)']
    
    rm = (rm_x, rm_y) # coordenadas da posição Rm
    rc = (rc_x, rc_y) # coordenadas da posição Rc
    
    return gamma_index_function(rc, rm, float(row['Dose Calculada (Dc)']), float(row['Dose Medida (Dm)']), max_Dm)

## *Medidas de Agosto/2022*

* Abaixo se encontra a análise dos 25 medidos realizadas em agosto/2022, onde foram calculados os valores de gamma-index dos 49 pontos calculados sobre e na vizinhança de cada ponto medido,
* Também foi feita a análise dos resultados a partir dos valores de menor gamma-index destes valores.

### Resultados - 1ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_1 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 1 - Agosto.csv'
df_augustData_sim_1 = pd.read_csv(august_data_sim_1)

# Informações do Dataframe dos dados
print(df_augustData_sim_1.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_1['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_sim_1 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_1), 49):
    group_df = df_augustData_sim_1.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_sim_1['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_sim_1.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_sim_1.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_sim_1)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 1ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_1 = pd.DataFrame(list(gamma_index_sim_1.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_1.head(25))

### Resultados - 2ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_2 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 2 - Agosto.csv'
df_augustData_sim_2 = pd.read_csv(august_data_sim_2)

# Informações do Dataframe dos dados
print(df_augustData_sim_2.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_2['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_sim_2 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_2), 49):
    group_df = df_augustData_sim_2.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_sim_2['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_sim_2.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_sim_2.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_sim_2)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 2ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_2 = pd.DataFrame(list(gamma_index_sim_2.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_2.head(25))

### Resultados - 3ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_3 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 3 - Agosto.csv'
df_augustData_sim_3 = pd.read_csv(august_data_sim_3)

# Informações do Dataframe dos dados
print(df_augustData_sim_3.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_3['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_sim_3 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_3), 49):
    group_df = df_augustData_sim_3.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_sim_3['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_sim_3.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_sim_3.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_sim_3)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 3ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_3 = pd.DataFrame(list(gamma_index_sim_3.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_3.head(25))

### Resultados - 4ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_4 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 4 - Agosto.csv'
df_augustData_sim_4 = pd.read_csv(august_data_sim_4)

# Informações do Dataframe dos dados
print(df_augustData_sim_4.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_4['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_sim_4 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_4), 49):
    group_df = df_augustData_sim_4.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_sim_4['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_sim_4.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_sim_4.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_sim_4)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 4ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_4 = pd.DataFrame(list(gamma_index_sim_4.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_4.head(25))

### Resultados - 5ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_5 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 5 - Agosto.csv'
df_augustData_sim_5 = pd.read_csv(august_data_sim_5)

# Informações do Dataframe dos dados
print(df_augustData_sim_5.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_5['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_sim_5 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_5), 49):
    group_df = df_augustData_sim_5.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_sim_5['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_sim_5.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_sim_5.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_sim_5)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 5ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_5 = pd.DataFrame(list(gamma_index_sim_5.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_5.head(25))

### Resultados - 6ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_6 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 6 - Agosto.csv'
df_augustData_sim_6 = pd.read_csv(august_data_sim_6)

# Informações do Dataframe dos dados
print(df_augustData_sim_6.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_6['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_sim_6 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_6_sim_6), 49):
    group_df = df_augustData_sim_6.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_sim_6['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_sim_6.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_sim_6.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_sim_6)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 6ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_6 = pd.DataFrame(list(gamma_index_sim_6.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_6.head(25))

### Resultados - 7ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_7 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 1 - Agosto.csv'
df_augustData_sim_7 = pd.read_csv(august_data_sim_7)

# Informações do Dataframe dos dados
print(df_augustData_sim_7.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_7['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_sim_7 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_7), 49):
    group_df = df_augustData_sim_7.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_sim_7['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_sim_7.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_sim_7.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_sim_7)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 7ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_7 = pd.DataFrame(list(gamma_index_sim_7.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_7.head(25))

## *Medidas de Fevereiro/2023*

* Abaixo se encontra a análise dos 25 medidos realizadas em fevereiro/2023, onde foram calculados os valores de gamma-index dos 49 pontos calculados sobre e na vizinhança de cada ponto medido,
* Também foi feita a análise dos resultados a partir dos valores de menor gamma-index destes valores.

### Resultados - 1ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_1 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 1 - Fevereiro.csv'
df_februaryData_sim_1 = pd.read_csv(february_data_sim_1)

# Informações do Dataframe dos dados
print(df_februaryData_sim_1.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_1['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_1 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_1), 49):
    group_df = df_februaryData_sim_1.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_1['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_1.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_1.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_1)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 1ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_1 = pd.DataFrame(list(gamma_index_feb_1.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_1.head(25))

### Resultados - 2ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_2 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 2 - Fevereiro.csv'
df_februaryData_sim_2 = pd.read_csv(february_data_sim_2)

# Informações do Dataframe dos dados
print(df_februaryData_sim_2.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_2['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_2 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_2), 49):
    group_df = df_februaryData_sim_2.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_2['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_2.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_2.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_2)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 2ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_2 = pd.DataFrame(list(gamma_index_feb_2.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_2.head(25))

### Resultados - 3ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_3 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 3 - Fevereiro.csv'
df_februaryData_sim_3 = pd.read_csv(february_data_sim_3)

# Informações do Dataframe dos dados
print(df_februaryData_sim_3.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_3['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_3 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_3), 49):
    group_df = df_februaryData_sim_3.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_3['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_3.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_3.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_3)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 3ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_3 = pd.DataFrame(list(gamma_index_feb_3.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_3.head(25))

### Resultados - 4ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_4 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 4 - Fevereiro.csv'
df_februaryData_sim_4 = pd.read_csv(february_data_sim_4)

# Informações do Dataframe dos dados
print(df_februaryData_sim_4.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_4['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_4 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_4), 49):
    group_df = df_februaryData_sim_4.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_4['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_4.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_4.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_4)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 4ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_4 = pd.DataFrame(list(gamma_index_feb_4.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_4.head(25))

### Resultados - 5ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_5 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 5 - Fevereiro.csv'
df_februaryData_sim_5 = pd.read_csv(february_data_sim_5)

# Informações do Dataframe dos dados
print(df_februaryData_sim_5.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_5['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_5 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_5), 49):
    group_df = df_februaryData_sim_5.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_5['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_5.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_5.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_5)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 5ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_5 = pd.DataFrame(list(gamma_index_feb_5.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_5.head(25))

### Resultados - 6ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_6 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 6 - Fevereiro.csv'
df_februaryData_sim_6 = pd.read_csv(february_data_sim_6)

# Informações do Dataframe dos dados
print(df_februaryData_sim_6.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_6['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_6 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_6), 49):
    group_df = df_februaryData_sim_6.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_6['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_6.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_6.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_6)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 6ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_6 = pd.DataFrame(list(gamma_index_feb_6.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_6.head(25))

### Resultados - 7ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [ ]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_7 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 1 - Fevereiro.csv'
df_februaryData_sim_7 = pd.read_csv(february_data_sim_7)

# Informações do Dataframe dos dados
print(df_februaryData_sim_7.info())

#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [ ]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_7['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_7 = {}

In [ ]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_7), 49):
    group_df = df_februaryData_sim_7.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_7['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [ ]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_7.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_7.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_7)

# Calcular as porcentagens dos valores acima
porcentagem_maiores_que_um = (quantidade_menores_ou_iguais_que_um / total_valores) * 100

porcentagem_menores_que_um = (quantidade_maiores_que_um / total_valores) * 100

In [ ]:
# Imprimir os resultados
print("Valores referentes às medidas da 7ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a 1:", quantidade_menores_ou_iguais_a_1)
print("Quantidade de valores reprovados, ou seja, maiores que 1:", quantidade_maiores_que_1)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [ ]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_7 = pd.DataFrame(list(gamma_index_feb_7.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [ ]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_7.head(25))

## *Salvando os Resultados*

* Abaixo, os resultados são salvos em um arquivo no formato Excel, com os resultados de cada mês em uma aba diferente

In [ ]:
# Criando um arquivo 'resultados_analises_programa.xlsx' em formato Excel, que conterá duas abas, uma para cada DataFrame
with pd.ExcelWriter('resultados_analises_programa.xlsx', engine='xlsxwriter') as writer:
    # Salvando o primeiro DataFrame (Agosto)) em uma aba chamada 'Resultados - Agosto'
    df_augustData.to_excel(writer, sheet_name='Resultados - Agosto', index=False)

    # Salvando o segundo DataFrame (Fevereiro) em uma aba chamada 'Resultados - Fevereiro'
    df_februaryData.to_excel(writer, sheet_name='Resultados - Fevereiro', index=False)